# OpenAI Function Calling ✅ 


## Introduction
This notebook demonstrates how to use `OpenAI's API for function calling with the GPT-3.5-Turbo model`. We'll cover updating the code to use the new API and provide an example function for fetching current weather information.

#### Notes

- OpenAI has announced the release of an updated GPT-3.5-Turbo model and the deprecation of the `gpt-3.5-turbo-0613` model.
- The `openai.ChatCompletion.create` has been removed in OpenAI's Python client version 1.0.0 and later. The API has changed, requiring code updates to use the new format such as `client.chat.completions.create`.

### Setup the Environment, OpenAI API Key  and Imports
First, we need to import the necessary libraries and set up the OpenAI API key:

In [ ]:
# Setting Up OpenAI API
# Import necessary libraries
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ = load_dotenv(find_dotenv()) # read local .env file

# Set OpenAI API key
openai.api_key = os.environ['OPENAI_API_KEY']  # Replace with your actual API key

# Print OpenAI API key (masked)
print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')[:5]}*****")



OPENAI_API_KEY: sk-ft*****


**Note** Ensure you have the required packages installed:
```py
%pip install rich
```
The rich library helps to improve the readability of nested dictionary outputs.

In [3]:
# Import the necessary modules from rich
from rich import print
from rich.pretty import Pretty

## Code Examples

### Example: Getting Current Weather

We will use an example of getting the current weather to demonstrate function calling. This is a good example because it involves external data that the language model can't generate on its own.

In [4]:
import json

# Example dummy function hard coded to return the same weather
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

#### Define the Function for OpenAI API

In [5]:
# Define the function for OpenAI API
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

#### Example User Message

In [6]:
# Example user message
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Amsterdam?"
    }
]

#### Call the ChatCompletion Endpoint

In [7]:
# Call the ChatCompletion endpoint using the new API format
client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])  # Replace with your actual API key

# Call the ChatCompletion endpoint using the new API format
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions  # If you're using function calling
)

print(response)


ChatCompletion(
    id='chatcmpl-AxD93iIuf9pyL6BsnmhijpTQXcFLj',
    choices=[
        Choice(
            finish_reason='function_call',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                audio=None,
                function_call=FunctionCall(arguments='{"location":"Amsterdam"}', name='get_current_weather'),
                tool_calls=None
            )
        )
    ],
    created=1738675245,
    model='gpt-3.5-turbo-0125',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=17,
        prompt_tokens=82,
        total_tokens=99,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

**Explanation output** This output represents a response from OpenAI's GPT-3.5-Turbo model. The model was prompted with a message, and its response includes a function call to get_current_weather with the argument {"location":"Amsterdam"}. The response indicates that the `assistant decided to call the function instead of generating text content. The response includes metadata such as the model used, creation time, and token usage details`.

#### Access the Message Content

To access response as an object, we need to use `dot notation`

In [8]:
# Extract the content from the response
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    audio=None,
    function_call=FunctionCall(arguments='{"location":"Amsterdam"}', name='get_current_weather'),
    tool_calls=None
)

In [9]:
# Convert response object to dictionary
response_dict = response.model_dump()

# Pretty print
formatted_response = json.dumps(response_dict, indent=4)
print(formatted_response)


{
    "id": "chatcmpl-AxD93iIuf9pyL6BsnmhijpTQXcFLj",
    "choices": [
        {
            "finish_reason": "function_call",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "audio": null,
                "function_call": {
                    "arguments": "{\"location\":\"Amsterdam\"}",
                    "name": "get_current_weather"
                },
                "tool_calls": null
            }
        }
    ],
    "created": 1738675245,
    "model": "gpt-3.5-turbo-0125",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 17,
        "prompt_tokens": 82,
        "total_tokens": 99,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0
        }
    }
}

**Explanation output** This output is from an OpenAI GPT-3.5-Turbo model, indicating that it received a prompt and decided to make a function call. The specific function called is get_current_weather with the argument {"location":"Amsterdam"}. The response includes metadata such as the response ID, the model used, creation time, token usage, and details about the function call. `The message generated by the assistant does not contain direct content but initiates a function call to gather the required information.`

- Access the Message Content 

In [10]:
# Extract the content from the response
response_message = response.choices[0].message
content = response_message.content  # Use dot notation
print(content)


None

#### Handling Function Calls

- Handling Function Calls
If the response contains a function call (as seen in the original output), content will be None, and you should `access the function details` instead:
`response_message["function_call"] has been replaced`

In [11]:
if response_message.function_call:
    function_name = response_message.function_call.name
    function_args = response_message.function_call.arguments
    print(f"Function Name: {function_name}")
    print(f"Arguments: {function_args}")
else:
    print(f"Assistant Response: {response_message.content}")


Function Name: get_current_weather

Arguments: {"location":"Amsterdam"}

#### Convert Response Object to a Dictionary

In [12]:
# Convert response object to a dictionary
response_dict = response_message.model_dump()
print(response_dict)

{
    'content': None,
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': {'arguments': '{"location":"Amsterdam"}', 'name': 'get_current_weather'},
    'tool_calls': None
}

**Explanation output** This output represents part of a response from an AI assistant (role: "assistant") indicating that it has chosen to call a function rather than generate direct content. The function being called is `get_current_weather` with the argument `{"location":"Amsterdam"}`. The fields `content, refusal, audio, and tool_calls are null`, indicating that no textual content, refusal message, audio response, or tool calls were included in the response.

In [ ]:
# Access function call details using dot notation
arguments_json = response_message.function_call.arguments  # This is already a string

# Parse JSON string into a Python dictionary
arguments_dict = json.loads(arguments_json)

print(arguments_dict)

{'location': 'Amsterdam'}

In [ ]:
# Parse the JSON string in the 'arguments' field of the function_call attribute of the response_message object into a Python dictionary.
args = json.loads(response_message.function_call.arguments)

In [15]:
get_current_weather(args)

'{"location": {"location": "Amsterdam"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

* Pass a message that is not related to a function.

In [16]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [17]:
# Call the ChatCompletion endpoint using the new API format
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions  # If you're using function calling
)

print(response)

ChatCompletion(
    id='chatcmpl-AxD947YJbkET5KKzL6HOZoDakbM4c',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Hello! How can I assist you today?',
                refusal=None,
                role='assistant',
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1738675246,
    model='gpt-3.5-turbo-0125',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=11,
        prompt_tokens=76,
        total_tokens=87,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

* Pass additional parameters to force the model to use or not a function.

#### Pass a Message Not Related to a Function

In [18]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # If you're using function calling
    function_call="auto",
)
print(response)


ChatCompletion(
    id='chatcmpl-AxD95wvjMP6xVHmBTY8Aii2Mex9h3',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Hello! How can I assist you today?',
                refusal=None,
                role='assistant',
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1738675247,
    model='gpt-3.5-turbo-0125',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=11,
        prompt_tokens=76,
        total_tokens=87,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

* Use mode 'none' for function call.

#### Call the ChatCompletion Endpoint for Non-Function Message

In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # If you're using function calling
    function_call="none",
)
print(response)


ChatCompletion(
    id='chatcmpl-AxD95J3zxxgqEXvRFKJVJXZfKEW1r',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Hello! How can I assist you today?',
                refusal=None,
                role='assistant',
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1738675247,
    model='gpt-3.5-turbo-0125',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=10,
        prompt_tokens=77,
        total_tokens=87,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

* When the message should call a function and still uses mode 'none'.

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Amsterdam?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # If you're using function calling
    function_call="none",
)
print(response)


ChatCompletion(
    id='chatcmpl-AxD96YfH1skAhKOPTmSHHupGiLc0H',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='I can look up the current weather in Amsterdam for you. Just give me a moment to retrieve 
the information.',
                refusal=None,
                role='assistant',
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1738675248,
    model='gpt-3.5-turbo-0125',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=23,
        prompt_tokens=82,
        total_tokens=105,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

* Force calling a function.

In [21]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # If you're using function calling
    function_call={"name": "get_current_weather"},
)
print(response)


ChatCompletion(
    id='chatcmpl-AxD97gslaZEZetUvFv9UE4tzFU5oS',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                audio=None,
                function_call=FunctionCall(
                    arguments='{"location":"San Francisco, CA","unit":"celsius"}',
                    name='get_current_weather'
                ),
                tool_calls=None
            )
        )
    ],
    created=1738675249,
    model='gpt-3.5-turbo-0125',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=14,
        prompt_tokens=86,
        total_tokens=100,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

* Final notes.

In [22]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Amsterdam!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # If you're using function calling
    function_call={"name": "get_current_weather"},
)
print(response)


ChatCompletion(
    id='chatcmpl-AxD97uaGLxf7QkZ4UWUIfOA7L5dun',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                audio=None,
                function_call=FunctionCall(arguments='{"location":"Amsterdam"}', name='get_current_weather'),
                tool_calls=None
            )
        )
    ],
    created=1738675249,
    model='gpt-3.5-turbo-0125',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=7,
        prompt_tokens=92,
        total_tokens=99,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

In [23]:
# Extract the content from the response
messages.append(response.choices[0].message)  


In [24]:
args = json.loads(response.choices[0].message.function_call.arguments)  


In [25]:
observation = get_current_weather(args)  # Assuming `get_current_weather` is properly defined


In [26]:

response_message = response.choices[0].message  # Use dot notation

if response_message.function_call:
    args = json.loads(response_message.function_call.arguments)  # Convert JSON string to dictionary
    observation = get_current_weather(args)  # Call the function with extracted arguments
    print(observation)
else:
    print(f"Assistant Response: {response_message.content}")


{"location": {"location": "Amsterdam"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}

**Explanation output** This output represents weather information for the location "Amsterdam." It includes the current temperature of 72 degrees Fahrenheit and a weather forecast indicating "sunny" and "windy" conditions.

- Final Fixed & Formatted Code

In [27]:

response_message = response.choices[0].message  # Use dot notation

if response_message.function_call:
    # Convert function call arguments (JSON string) to a Python dictionary
    args = json.loads(response_message.function_call.arguments)
    
    # Call the function with extracted arguments
    observation = get_current_weather(args)

    # Pretty-print the output
    print("\n🟢 Function Call Detected:")
    print(f"🔹 Function Name: {response_message.function_call.name}")
    print(f"🔹 Arguments: {json.dumps(args, indent=4)}")  # Pretty print JSON arguments
    print("\n🌦️ Weather Observation Result:")
    print(json.dumps(observation, indent=4))  # Assuming the function returns a dictionary
else:
    print("\n💬 Assistant Response:")
    print(response_message.content)


🟢 Function Call Detected:

🔹 Function Name: get_current_weather

🔹 Arguments: {
    "location": "Amsterdam"
}

🌦️ Weather Observation Result:

"{\"location\": {\"location\": \"Amsterdam\"}, \"temperature\": \"72\", \"unit\": \"fahrenheit\", \"forecast\": 
[\"sunny\", \"windy\"]}"

If the assistant calls get_current_weather with:

In [28]:

response_message = response.choices[0].message  # Use dot notation

if response_message.function_call:
    # Convert function call arguments (JSON string) to a Python dictionary
    args = json.loads(response_message.function_call.arguments)
    
    # Call the function with extracted arguments
    observation = get_current_weather(args)

    # Ensure `observation` is a dictionary, not a string
    if isinstance(observation, str):
        observation = json.loads(observation)

    # Pretty-print the function call details
    print("\n🟢 Function Call Detected:")
    print(f"🔹 Function Name: {response_message.function_call.name}")
    print(f"🔹 Arguments: {json.dumps(args, indent=4)}")

    # Pretty-print the weather observation
    print("\n🌦️ Weather Observation Result:")
    print(json.dumps(observation, indent=4))

else:
    print("\n💬 Assistant Response:")
    print(response_message.content)


🟢 Function Call Detected:

🔹 Function Name: get_current_weather

🔹 Arguments: {
    "location": "Amsterdam"
}

🌦️ Weather Observation Result:

{
    "location": {
        "location": "Amsterdam"
    },
    "temperature": "72",
    "unit": "fahrenheit",
    "forecast": [
        "sunny",
        "windy"
    ]
}

In [29]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [30]:

# Ensure observation is a properly formatted string
if isinstance(observation, dict):
    observation_str = json.dumps(observation)  # Convert dictionary to JSON string
else:
    observation_str = observation  # If already a string, use as-is

# Append the function result to the messages list
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": observation_str,  # Now a properly formatted string
    }
)

In [31]:

# Initialize OpenAI client (replace with your actual API key)
client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])  # Replace with your actual API key

# Existing messages list
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the weather like in Amsterdam?"}
]

# Call OpenAI API to get response
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

# Extract the assistant's response
response_message = response.choices[0].message

if response_message.function_call:
    # Extract and parse function call arguments
    args = json.loads(response_message.function_call.arguments)
    
    # Simulating the function call (replace with real implementation)
    observation = get_current_weather(args)

    # Ensure observation is a JSON string before appending to messages
    if isinstance(observation, dict):
        observation_str = json.dumps(observation)  # Convert dict to JSON string
    else:
        observation_str = str(observation)  # Ensure it's a string

    # Append the function result back to the conversation history
    messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation_str,  # ✅ Now always a string
        }
    )

    # Print formatted function call details
    print("\n🟢 Function Call Detected:")
    print(f"🔹 Function Name: {response_message.function_call.name}")
    print(f"🔹 Arguments: {json.dumps(args, indent=4)}")

    # Convert observation back to a dictionary if needed for pretty-printing
    if isinstance(observation_str, str):
        observation = json.loads(observation_str)

    # Print formatted weather observation
    print("\n🌦️ Weather Observation Result:")
    print(json.dumps(observation, indent=4))

else:
    print("\n💬 Assistant Response:")
    print(response_message.content)

# Call OpenAI API again with updated messages
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

# Pretty-print the final OpenAI response
formatted_response = json.dumps(response.model_dump(), indent=4)
print("\n🔵 Final OpenAI Response:")
print(formatted_response)


💬 Assistant Response:

I currently do not have real-time information on weather. I recommend checking a weather website or app for the 
most up-to-date weather information in Amsterdam.

🔵 Final OpenAI Response:

{
    "id": "chatcmpl-AxD9Bp1LtmShigBQl6GpfodtjpwO8",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "I'm sorry, but I am not able to provide real-time weather information. I recommend 
checking a reliable weather website or using a weather app on your phone for the most up-to-date information on the
weather in Amsterdam.",
                "refusal": null,
                "role": "assistant",
                "audio": null,
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1738675253,
    "model": "gpt-3.5-turbo-0125",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 45,
        "prompt_tokens": 25,
        "total_tokens": 70,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0
        }
    }
}

**Explantion output** The issue is that the assistant is not calling the function (get_current_weather). Instead, it is giving a default response, `saying it does not have real-time information`.
🔹 Why Is This Happening?  
- The assistant isn’t aware that it can call a function.  
- You need to explicitly define functions in your request to OpenAI.  
- No function definition was provided to OpenAI's API.  
- You must include functions in client.chat.completions.create().  
- function_call behavior isn’t specified.  
- OpenAI won’t automatically call a function unless you tell it to.

#### ✅ Final Fixed & Optimized Code

Let's fix the above code in order to get a significant response from the assistent. Thus, in this case, get an answer about the actual weather in Amsterdam. 

In [32]:

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])  # Replace with your actual API key

# Define the function OpenAI can call
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather for a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., 'Boston'"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location", "unit"]
        }
    }
]

# Conversation messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the weather like in Amsterdam?"}
]

# Call OpenAI API with function definition
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # ✅ Add function definition
    function_call="auto"  # ✅ Force the model to call a function if needed
)

# Extract the assistant's response
response_message = response.choices[0].message

if response_message.function_call:
    # Extract and parse function call arguments
    args = json.loads(response_message.function_call.arguments)
    
    # Simulating the function call (replace with real implementation)
    observation = get_current_weather(args)

    # Ensure observation is a JSON string before appending to messages
    if isinstance(observation, dict):
        observation_str = json.dumps(observation)
    else:
        observation_str = str(observation)

    # Append function response back to messages
    messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation_str,
        }
    )

    # Print formatted function call details
    print("\n🟢 Function Call Detected:")
    print(f"🔹 Function Name: {response_message.function_call.name}")
    print(f"🔹 Arguments: {json.dumps(args, indent=4)}")

    # Pretty-print the weather observation
    print("\n🌦️ Weather Observation Result:")
    print(json.dumps(json.loads(observation_str), indent=4))

else:
    print("\n💬 Assistant Response:")
    print(response_message.content)

# Call OpenAI API again with updated messages to continue the conversation
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # ✅ Keep functions defined for future calls
)

# Pretty-print the final OpenAI response
formatted_response = json.dumps(response.model_dump(), indent=4)
print("\n🔵 Final OpenAI Response:")
print(formatted_response)


🟢 Function Call Detected:

🔹 Function Name: get_current_weather

🔹 Arguments: {
    "location": "Amsterdam",
    "unit": "celsius"
}

🌦️ Weather Observation Result:

{
    "location": {
        "location": "Amsterdam",
        "unit": "celsius"
    },
    "temperature": "72",
    "unit": "fahrenheit",
    "forecast": [
        "sunny",
        "windy"
    ]
}

🔵 Final OpenAI Response:

{
    "id": "chatcmpl-AxD9DjeipyzaTkCcc819VmUeENV4d",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "The current weather in Amsterdam is 72\u00b0F with sunny and windy conditions.",
                "refusal": null,
                "role": "assistant",
                "audio": null,
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1738675255,
    "model": "gpt-3.5-turbo-0125",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 17,
        "prompt_tokens": 137,
        "total_tokens": 154,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0
        }
    }
}

**Explanation output** To sum up, now, `the assistent gives the right answer`, thus info about the actual weather in Amsterdam. Thus, the final OpenAI Response: "message":   {"content": "The current weather in Amsterdam is 22\u00b0C (72\u00b0F) with sunny and windy`}. What it has been fixed and optimized, compared to pthe previous output, has been: 
- Uses dot notation instead of response["choices"][0]["message"]  
- Ensures observation is properly formatted (json.dumps(observation))  
- Handles function calls correctly  
- Prints a structured, readable response  

## Conclusion
This notebook demonstrates how to integrate OpenAI's function calling capabilities using the updated GPT-3.5-Turbo model. By following these examples, you can effectively utilize the latest OpenAI API changes in your projects.